# Lab 3: Distilling the Web into a GIS Dataset
## Daniel Kim

My website: http://students.washington.edu/dtm96/sam_map.html

Datasource: https://www.cnbc.com/2018/01/12/heres-a-map-of-where-walmart-is-closing-more-than-60-sams-club-stores.html

I used an article that listed out the addresses of all the Sam's Clubs that were closing in the United States.

The intellectual significance is that it is very useful for anyone who wants to know visually where the sam's clubs are located. This map could be used as a visual aid in presentaitons to managers and customers of where the Sam's clubs are closing.

This was a challenging project for me personally since I did not have any python experience previously and I went off of all the vague explanations of stackoverflow and the like. But by continuously looking forminformation and clarification on the specs and on online resources, I was able to finally figure out how to map the addresses.
                                                                                                                                        
Hours Spent in total: 20 hours
- scraping: 5 hours
- Geocoding: 4 hours
- Geodataframe: 6 hours
- Mapping: 2 hours
- Converting to shapefile: 3 hours

In [1]:
# Import packages for scraping and exporting csv
import cssselect, urllib2, lxml, ssl, csv, geopy, shapely, fiona, folium, json
from lxml import html
from geopandas import GeoDataFrame
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd


In [2]:
# Request HTML by using Magic Browser parameter (codes are provided from class sample)
url = "https://www.cnbc.com/2018/01/12/heres-a-map-of-where-walmart-is-closing-more-than-60-sams-club-stores.html"
req = urllib2.Request(url, headers={'User-Agent' : "Magic Browser"}) 
con = urllib2.urlopen(req)
doc_text = con.read()
doc = lxml.html.fromstring(doc_text)
doc.make_links_absolute(url)

In [3]:
# Extract content of website 
addlist = [] # create list to extract addresses from website
for row in doc.cssselect(".group ul li"):
        add = row.text_content()
        addlist.append(row)

# get rid of unwanted text from list
for x in range(0,3):
    addlist.pop(0)

# keep list of coordinates
latlist = []
longlist = []

from geopy.geocoders import Nominatim
geolocator = Nominatim()

# aquire lat and longitudes of addresses
# source: https://pypi.python.org/pypi/geopy
for row in addlist:
    try:
        location = geolocator.geocode([row])
        latlist.appened(location.latitude)
        longlist.append(location.longitude)
    except:
        street, state, ST = row.text.split(',')
        location = geolocator.geocode(state)
        latlist.append(location.latitude)
        longlist.append(location.longitude)

# convert address html style into text for better readability
addresslist = []
for text in addlist:
    addresslist.append(text.text)

# compile address and coordinates into dictionary
add = {'ADDRESS': addresslist, 'LATITUDE': latlist, 'LONGITUDE': longlist}

# see if data is in dictionary
# print add

In [4]:
# export data as csv
sortadd = sorted(add.keys())
with open('Sams_Club_locations.csv', 'wb') as output_file:
    writer = csv.writer(output_file)
    writer.writerow(sortadd)
    writer.writerows(zip(*[add[key] for key in sortadd]))
    

In [5]:
# Can Create a dataframe with pandas
# df = pd.DataFrame(add, columns=['ADDRESS', 'LATITUDE', 'LONGITUDE'])

# another way to do it...
# df = pd.read_csv('Sams_Club_locations.csv')
# df

In [6]:
# read exported csv file
df = pd.read_csv('Sams_Club_locations.csv')

# zip coordinates to convert to points
# stackoverflow
coordinates = zip(df['LONGITUDE'], df['LATITUDE'])

# convert to coordinates
geometry = [shapely.geometry.Point(coordinate)for coordinate in coordinates]
geometrylist = gpd.GeoSeries(geometry)

# Create Geodataframe
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=fiona.crs.from_epsg(4326))

# show geodataframe
gdf

,ADDRESS,LATITUDE,LONGITUDE,geometry
0,"8801 Old Seward Hwy, Anchorage, AK",61.216313,-149.894852,POINT (-149.8948523 61.2163129)
1,"1074 N Muldoon Rd, Anchorage, AK",61.216313,-149.894852,POINT (-149.8948523 61.2163129)
2,"48 College Rd, Fairbanks, AK",64.837845,-147.716675,POINT (-147.716675 64.837845)
3,"3900 Grants Mill Rd, Irondale, AL",33.538160,-86.707210,POINT (-86.70721020000001 33.5381601)
4,"2425 E Florence Blvd, Casa Grande, AZ",32.879382,-111.757561,POINT (-111.7575614 32.8793816)
5,"5757 E State Route 69, Prescott Valley, AZ",34.648897,-112.280214,POINT (-112.280213830875 34.6488975)
6,"1375 S Arizona Ave, Chandler, AZ",33.306713,-111.840849,POINT (-111.8408489 33.3067132)
7,"15255 N Northsight Blvd, Scottsdale, AZ",33.509121,-111.899237,POINT (-111.8992365 33.5091215)
8,"3360 El Camino Ave, Sacramento, CA",38.581572,-121.494400,POINT (-121.4943996 38.5815719)
9,"17835 Gale Ave, City of Industry, CA",34.060457,-118.009033,POINT (-118.009033449214 34.0604565)


In [7]:
# render folium map
sam_map = folium.Map(location=(39.0902, -99.7129), 
                     zoom_start=4,
                    tiles="Stamen Toner")

# convert Geodataframe into geojson for map visualization
geom = folium.features.GeoJson(gdf.to_json()) 
sam_map.add_child(geom)

In [8]:
# saves map as html
sam_map.save("sam_map.html")

In [ ]:
# save GeoDataFrame as a shapefile
gdf.to_file("sams_map.shp", driver = "ESRI Shapefile")